In [10]:
import pybamm;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;
import matplotlib as mpl; 
fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import multiprocessing
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [11]:
V_max = 4.2;        V_min = 2.5;
exp  = pybamm.Experiment(
    [ (
        f"Discharge at 1 C until {V_min} V", 
        f"Charge at 1 C until {V_max} V", 
        f"Hold at {V_max} V until C/5")    ] * 1 )  
Model_0  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "sol full"  ,
    }) 
Model_1  = pybamm.lithium_ion.DFN(options={
    "solvent diffusion": "double spatial consume w refill",
    "electrolyte conductivity": "sol full"  ,
    }) 

In [12]:
def EC_diffusivity_5E_5(c_e, c_EC , T):
    D_ec_dim = (
        (c_EC >= 0 ) * 5e-5 
        +  (c_EC < 0 ) * 0 
    )
    return D_ec_dim
def EC_transference_number_0p7(c_e,c_EC, T):# Mark Ruihe add update 221212
    c_EC_0 = pybamm.Parameter("Typical EC concentration [mol.m-3]")
    Xi = ( 
        (c_EC < 0 ) * 0 
        + 
        (c_EC <= c_EC_0) * (c_e >= 0) * (0.7 * c_EC / c_EC_0 )
        +
        (c_EC > c_EC_0 ) * 0.7  
    )
    return Xi
Para_0=pybamm.ParameterValues("Li2023_ECdrag")
Para_0.update({'Negative electrode porosity':0.05})
Para_0.update({"EC Lithium ion cross diffusivity [m2.s-1]":1.5e-12})
c_e = Model_0.variables["Electrolyte concentration [mol.m-3]"]
c_EC= Model_0.variables["EC concentration [mol.m-3]"]
T = Model_0.variables["Cell temperature [K]"]
D_e = Para_0["Electrolyte diffusivity [m2.s-1]"]
sigma_e = Para_0["Electrolyte conductivity [S.m-1]"]
Xi = Para_0["EC transference number"]
Model_0.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
Model_0.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
Model_0.variables["EC transference number"] = Xi(c_e,c_EC, T)

Para_1=pybamm.ParameterValues("Li2023_ECdrag")
Para_1.update({'Negative electrode porosity':0.05})
Para_1.update({"EC Lithium ion cross diffusivity [m2.s-1]":1.5e-12})
Para_1.update({"EC transference number":EC_transference_number_0p7})
c_e = Model_1.variables["Electrolyte concentration [mol.m-3]"]
c_EC= Model_1.variables["EC concentration [mol.m-3]"]
T = Model_1.variables["Cell temperature [K]"]
D_e = Para_1["Electrolyte diffusivity [m2.s-1]"]
sigma_e = Para_1["Electrolyte conductivity [S.m-1]"]
Xi = Para_1["EC transference number"]
Model_1.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e,c_EC, T)
Model_1.variables["Electrolyte conductivity [S.m-1]"] = sigma_e(c_e,c_EC, T)
Model_1.variables["EC transference number"] = Xi(c_e,c_EC, T)

In [13]:
Sim_0    = pybamm.Simulation(
    Model_0, experiment = exp,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)       
Sol_0    = Sim_0.solve()

In [14]:
Sim_1    = pybamm.Simulation(
    Model_1, experiment = exp,
    parameter_values = Para_1,
    solver = pybamm.CasadiSolver(),)       
Sol_1    = Sim_1.solve()

In [15]:
label = ["DD Xi=0.85","DD Xi=0.7",] 
output_variables3 = [
    "Terminal voltage [V]", 
    "Electrolyte potential [V]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "EC transference number",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]"
]
quick_plot = pybamm.QuickPlot(
    [Sol_0,Sol_1], output_variables3,label,variable_limits='fixed',
    time_unit='hours',
    spatial_unit='mm',     #  (“m”, “mm”, or “um”)
    n_rows=2) #figsize = (18,12),
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.2678828302160823, step=0.01267882830216082…